## Codice per la valutazione di circuito ideale e rumoroso

Il seguente codice si occupa di generare il circuito risultato migliore dalle valutazioni precedenti e simularlo in un contesto sia ideale che rumoroso tramite l'uso delle primitive AER messe a disposizione dalla libreria  qiskit. Per qualunque valutazione ed info più specifica non contenuta nei seguenti markdown si rimanda alla relazione.

In [ ]:
# Installazione delle librerie necessarie
%pip install qiskit qiskit-aer qiskit-ibm-runtime qiskit-algorithms qiskit-machine-learning
%pip install numpy pandas scikit-learn matplotlib scipy


In [ ]:
# Import delle librerie
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from qiskit import transpile
from qiskit_aer import AerSimulator #cosi so che qui ho primitive per backend reale, non so dove gira, ma so che gira su uno reale 
                                    #e gli posso mettere rumore se voglio
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit.primitives import StatevectorSampler
from qiskit.circuit.library import ZFeatureMap
from qiskit.circuit.library import EfficientSU2
from scipy.optimize import minimize


# Funzioni che preparano il dataset in maniera classica, e come fatto per il resto della trattazione

In [ ]:
# Caricamento e preparazione del dataset Wine
def load_and_prepare_wine_dataset():
    """Carica il dataset Wine e applica PCA a 5 componenti """
    
    # Caricamento dataset Wine
    wine = load_wine()
    X = wine.data
    y = wine.target

    # Standardizzazione
    scaler_standard = StandardScaler()
    X_scaled = scaler_standard.fit_transform(X)

    # Applicazione PCA a 5 componenti
    pca = PCA(n_components=5)
    X_pca = pca.fit_transform(X_scaled)

    return X_pca, y  # Restituisce y originale con 3 classi

# Funzione di normalizzazione per ZFeatureMap
def rescaler_z_feature_map(X):
    """Normalizza le feature per ZFeatureMap"""
    scaler = MinMaxScaler()
    return scaler.fit_transform(X)


# Split del dataset ed istanziazione componenti del circuito

In [ ]:
# Caricamento e preparazione dei dati
X, y = load_and_prepare_wine_dataset()
X_normalized = rescaler_z_feature_map(X)

# Suddivisione train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.3, random_state=42, stratify=y
)

# Conversione in liste per compatibilità
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()
y_train_list = y_train.tolist()
y_test_list = y_test.tolist()


# Configurazione del circuito quantistico
num_features = 5  


feature_map = ZFeatureMap(feature_dimension=num_features, reps=1)
feature_map.barrier()

ansatz = EfficientSU2(num_features, reps=1)
ansatz.barrier()

# Combinazione di encoding e ansatz
vqc_circuit = feature_map.compose(ansatz)
vqc_circuit.measure_all()



# Logica classica di gestione delle probabilità associate alla classificazione delle istanze

L'unica differenza rispetto le considerazioni affrontate nel resto della trattazione consiste nella separazione del metodo "classification_probability" in:
* classification_probability_ideal(data, variational_params) => gestisce il caso reale sfruttando StateVector come Sampler
* classification_probability_noisy(data, variational_params) => gestisce il caso rumoroso utilizzando un simulatore di back-end realistico (in questo caso, FakeVigoV2, che imita un dispositivo IBM reale).

In [ ]:

def circuit_instance(features, variational_params):
    """Crea un'istanza del circuito con parametri specifici"""
    parameters = {}
    # Assegnazione parametri della feature map
    for i, p in enumerate(feature_map.ordered_parameters):
        parameters[p] = features[i]
    # Assegnazione parametri dell'ansatz
    for i, p in enumerate(ansatz.ordered_parameters):
        parameters[p] = variational_params[i]
    return vqc_circuit.assign_parameters(parameters)

def interpreter(bitstring):
    """Interpreta un bitstring come etichetta di classe """
    hamming_weight = sum(int(bit) for bit in bitstring)
    return hamming_weight % 3  

def label_probability(measurement_results):
    """Calcola le probabilità delle classi dai risultati di misura """
    total_shots = sum(measurement_results.values())
    probabilities = {0: 0, 1: 0, 2: 0}  

    for bitstring, counts in measurement_results.items():
        label = interpreter(bitstring)
        probabilities[label] += counts / total_shots

    return probabilities

# Funzioni di classificazione (ideale e con rumore)
def classification_probability_ideal(data, variational_params):
    """Classificazione in ambiente ideale (senza rumore)"""
    circuits = [circuit_instance(features, variational_params) for features in data]
    sampler = StatevectorSampler()
    results = sampler.run(circuits).result()
    classifications = [
        label_probability(results[i].data.meas.get_counts())
        for i in range(len(circuits))
    ]
    return classifications

def classification_probability_noisy(data, variational_params):
    """Classificazione con rumore quantistico"""
    circuits = [circuit_instance(features, variational_params) for features in data]

    fake_backend = FakeVigoV2()
    sim = AerSimulator.from_backend(fake_backend)

    transpiled_circuits = transpile(circuits, sim)

    results = sim.run(transpiled_circuits).result()

    classifications = []
    for result in results.get_counts():
        classifications.append(label_probability(result))

    return classifications


# Due funzioni per gestire il caso ideale ed rumoroso

Le due funzioni cost_function_ideal e cost_function_noisy calcolano la funzione di costo (loss function) da minimizzare durante l’addestramento, una chiama la funzione ideale per il calcolo delle probabilità del blocco sopra, l'altra quella rumorosa


In [ ]:
# Funzioni di costo per l'ottimizzazione 

def cost_function_ideal(data, labels, variational_params):
    """Funzione di costo in ambiente ideale"""
    classifications = classification_probability_ideal(data, variational_params)
    # Converte le probabilità in formato compatibile con log_loss per 3 classi
    y_pred_proba = [[p[0], p[1], p[2]] for p in classifications]
    cost = log_loss(
        y_true=labels,
        y_pred=y_pred_proba,
        labels=[0, 1, 2]  
    )
    return cost

def cost_function_noisy(data, labels, variational_params):
    """Funzione di costo con rumore quantistico"""
    classifications = classification_probability_noisy(data, variational_params)
    # Converte le probabilità in formato compatibile con log_loss per 3 classi
    y_pred_proba = [[p[0], p[1], p[2]] for p in classifications]
    cost = log_loss(
        y_true=labels,
        y_pred=y_pred_proba,
        labels=[0, 1, 2]  
    )
    return cost

# _Questo è un logger che stampa ogni 5 iterazioni il valore di funzione obiettivo che il modello sta valutando. Utile anche per il resto del codice per stampare i grafici delle due log-loss (decommentare assieme alle istruzioni commentate nelle righe successive per utilizzarlo)_

In [ ]:
# # Logger per tracciare l'ottimizzazione
# class OptimizationLogger:
#     def __init__(self, use_noisy=False):
#         self.evaluations = 1
#         self.parameters = []
#         self.costs = []
#         self.use_noisy = use_noisy

#     def callback(self, xk):
#         if self.use_noisy:
#             cost = cost_function_noisy(X_train_list, y_train_list, xk)
#         else:
#             cost = cost_function_ideal(X_train_list, y_train_list, xk)

#         self.parameters.append(xk.copy())
#         self.costs.append(cost)

#         if self.evaluations % 5 == 0:  # Stampa ogni 5 iterazioni
#             print(f"Iterazione {self.evaluations}: Loss = {cost:.4f}")

#         self.evaluations += 1


In [ ]:

# Funzione obiettivo per l'ottimizzazione
def objective_function_ideal(variational_params):
    return cost_function_ideal(X_train_list, y_train_list, variational_params)

def objective_function_noisy(variational_params):
    return cost_function_noisy(X_train_list, y_train_list, variational_params)

# Scelta degli starting point ed avvio della minimizzazione della funzione obiettivo tramite cobyla

In [ ]:
# OTTIMIZZAZIONE DEL MODELLO IDEALE
print("\n" + "="*50)
print("OTTIMIZZAZIONE MODELLO IDEALE")
print("="*50)

# logger_ideal = OptimizationLogger(use_noisy=False)

np.random.seed(123)  # Seed diverso per ogni modello
initial_point_ideal = np.random.uniform(0, 2*np.pi, size=ansatz.num_parameters)

print(f"Numero di parametri da ottimizzare: {ansatz.num_parameters}")
print("Avvio ottimizzazione...")

result_ideal = minimize(
    objective_function_ideal,
    initial_point_ideal,
    method="COBYLA",
    options={"maxiter": 3000}, 
    # callback=logger_ideal.callback
)

optimal_params_ideal = result_ideal.x
print(f"Ottimizzazione completata. Loss finale: {result_ideal.fun:.4f}")


print("\n" + "="*50)
print("OTTIMIZZAZIONE MODELLO CON RUMORE")
print("="*50)

# logger_noisy = OptimizationLogger(use_noisy=True)
np.random.seed(456)
initial_point_noisy = np.random.uniform(0, 2*np.pi, size=ansatz.num_parameters)

print("Avvio ottimizzazione con rumore...")
result_noisy = minimize(
    objective_function_noisy,
    initial_point_noisy,
    method="COBYLA",
    options={"maxiter": 3000},
    # callback=logger_noisy.callback
)

optimal_params_noisy = result_noisy.x
print(f"Ottimizzazione completata. Loss finale: {result_noisy.fun:.4f}")



## Decommentare assieme al blocco precedente citato ed alle callback in quello appena sopra, per avere la stampa delle curve di convergenza della nostra funzione obiettivo

In [ ]:
# # Visualizzazione delle curve di convergenza
# plt.figure(figsize=(12, 5))

# plt.subplot(1, 2, 1)
# plt.plot(logger_ideal.costs, 'b-', label='Ideale', linewidth=2)
# plt.xlabel('Iterazioni')
# plt.ylabel('Loss')
# plt.title('Convergenza Modello Ideale')
# plt.grid(True, alpha=0.3)
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(logger_noisy.costs, 'r-', label='Con Rumore', linewidth=2)
# plt.xlabel('Iterazioni')
# plt.ylabel('Loss')
# plt.title('Convergenza Modello con Rumore')
# plt.grid(True, alpha=0.3)
# plt.legend()

# plt.tight_layout()
# plt.show()

# # Confronto delle curve di convergenza
# plt.figure(figsize=(10, 6))
# plt.plot(logger_ideal.costs, 'b-', label='Modello Ideale', linewidth=2)
# plt.plot(logger_noisy.costs, 'r-', label='Modello con Rumore', linewidth=2)
# plt.xlabel('Iterazioni')
# plt.ylabel('Loss')
# plt.title('Confronto Convergenza: Ideale vs Con Rumore')
# plt.grid(True, alpha=0.3)
# plt.legend()
# plt.show()




# Funzione per valutare le prestazioni del classificatore quantistico in base al tipo:

In [ ]:
def test_classifier(data, labels, variational_params, use_noisy=False):
    """Testa il classificatore e restituisce accuratezza e predizioni"""
    if use_noisy:
        probabilities = classification_probability_noisy(data, variational_params)
    else:
        probabilities = classification_probability_ideal(data, variational_params)

    predictions = [max(p, key=p.get) for p in probabilities]
    accuracy = sum(pred == true for pred, true in zip(predictions, labels)) / len(labels)

    return accuracy, predictions



# Valutazione delle accuratezze per i due modelli sfruttando i parametri ottimizzati

In [ ]:
# VALUTAZIONE DEI MODELLI
print("\n" + "="*50)
print("VALUTAZIONE DEI MODELLI")
print("="*50)

# Test modello ideale
print("\nModello Ideale:")
train_acc_ideal, train_pred_ideal = test_classifier(X_train_list, y_train_list, optimal_params_ideal, use_noisy=False)
test_acc_ideal, test_pred_ideal = test_classifier(X_test_list, y_test_list, optimal_params_ideal, use_noisy=False)

print(f"Accuratezza Train: {train_acc_ideal:.4f}")
print(f"Accuratezza Test: {test_acc_ideal:.4f}")

# Test modello con rumore
print("\nModello con Rumore:")
train_acc_noisy, train_pred_noisy = test_classifier(X_train_list, y_train_list, optimal_params_noisy, use_noisy=True)
test_acc_noisy, test_pred_noisy = test_classifier(X_test_list, y_test_list, optimal_params_noisy, use_noisy=True)

print(f"Accuratezza Train: {train_acc_noisy:.4f}")
print(f"Accuratezza Test: {test_acc_noisy:.4f}")




# Metriche varie grazie ad sk_learn e plot generali

In [ ]:

def calculate_metrics(y_true, y_pred, model_name):
    """Calcola e stampa metriche dettagliate per problema multi-classe"""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    print(f"\n{model_name}:")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")

    # Metriche per classe
    precision_per_class = precision_score(y_true, y_pred, average=None, zero_division=0)
    recall_per_class = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1_per_class = f1_score(y_true, y_pred, average=None, zero_division=0)

    print(f"  Metriche per classe:")
    for i in range(3):
        print(f"    Classe {i}: Prec={precision_per_class[i]:.4f}, Rec={recall_per_class[i]:.4f}, F1={f1_per_class[i]:.4f}")

    return accuracy, precision, recall, f1



In [ ]:
# Calcolo metriche per entrambi i modelli
print("\n" + "="*50)
print("METRICHE DETTAGLIATE SUL TEST SET")
print("="*50)

metrics_ideal = calculate_metrics(y_test_list, test_pred_ideal, "Modello Ideale")
metrics_noisy = calculate_metrics(y_test_list, test_pred_noisy, "Modello con Rumore")

# Creazione tabella di confronto delle metriche
metrics_comparison = pd.DataFrame({
    'Modello Ideale': metrics_ideal,
    'Modello con Rumore': metrics_noisy
}, index=['Accuracy', 'Precision ', 'Recall', 'F1-Score '])

print("\n" + "="*50)
print("TABELLA DI CONFRONTO METRICHE")
print("="*50)
print(metrics_comparison.round(4))

# Visualizzazione grafica delle metriche
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Grafico a barre delle metriche
metrics_comparison.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_title('Confronto Metriche di Performance ')
axes[0].set_ylabel('Valore')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)

# Heatmap delle metriche
sns.heatmap(metrics_comparison, annot=True, fmt='.4f', cmap='YlOrRd', ax=axes[1])
axes[1].set_title('Heatmap Metriche di Performance ')

plt.tight_layout()
plt.show()

# Confusion Matrix -
print("\n" + "="*50)
print("CONFUSION MATRICES ")
print("="*50)

cm_ideal = confusion_matrix(y_test_list, test_pred_ideal)
cm_noisy = confusion_matrix(y_test_list, test_pred_noisy)

print("\nConfusion Matrix - Modello Ideale:")
print(cm_ideal)
print("\nConfusion Matrix - Modello con Rumore:")
print(cm_noisy)

# Visualizzazione Confusion Matrices
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Etichette delle classi per le confusion matrix
class_names = ['Classe 0', 'Classe 1', 'Classe 2']

sns.heatmap(cm_ideal, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=class_names, yticklabels=class_names)
axes[0].set_title('Confusion Matrix - Modello Ideale')
axes[0].set_xlabel('Predetto')
axes[0].set_ylabel('Reale')

sns.heatmap(cm_noisy, annot=True, fmt='d', cmap='Reds', ax=axes[1],
            xticklabels=class_names, yticklabels=class_names)
axes[1].set_title('Confusion Matrix - Modello con Rumore')
axes[1].set_xlabel('Predetto')
axes[1].set_ylabel('Reale')

plt.tight_layout()
plt.show()

# Analisi dell'impatto del rumore
print("\n" + "="*50)
print("ANALISI IMPATTO DEL RUMORE")
print("="*50)

accuracy_drop = metrics_ideal[0] - metrics_noisy[0]
precision_drop = metrics_ideal[1] - metrics_noisy[1]
recall_drop = metrics_ideal[2] - metrics_noisy[2]
f1_drop = metrics_ideal[3] - metrics_noisy[3]

print(f"Degradazione dovuta al rumore:")
print(f"  Accuracy:  -{accuracy_drop:.4f} ({accuracy_drop/metrics_ideal[0]*100:.1f}%)")
print(f"  Precision: -{precision_drop:.4f} ({precision_drop/metrics_ideal[1]*100:.1f}%)")
print(f"  Recall:    -{recall_drop:.4f} ({recall_drop/metrics_ideal[2]*100:.1f}%)")
print(f"  F1-Score:  -{f1_drop:.4f} ({f1_drop/metrics_ideal[3]*100:.1f}%)")

# Analisi distribuzione delle predizioni per classe
print(f"\nDistribuzione predizioni sul test set:")
print(f"  Reali:          {np.bincount(y_test_list)}")
print(f"  Ideale:         {np.bincount(test_pred_ideal)}")
print(f"  Con rumore:     {np.bincount(test_pred_noisy)}")

print("\n" + "="*50)
print("ANALISI COMPLETATA")
print("="*50)